In [0]:
!pip install keras-bert
!pip install keras-xlnet
!pip install tokenization
!pip install bert
!pip install keras-lr-finder
!pip install bert-tensorflow
!pip install numpy==1.16.2

In [0]:
!nvidia-smi

Tue Mar 17 09:00:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.59       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    33W / 250W |  15767MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
%tensorflow_version 1.14
import os
# os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
# os.environ["CUDA_VISIBLE_DEVICES"] = '0'

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

import re
import string
import matplotlib.pyplot as plt
%matplotlib inline
import copy
import cv2
from os import listdir
from sklearn.metrics import accuracy_score
from PIL import Image

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import backend as K

sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) #command to run codeon multiple gpu

from keras_lr_finder import LRFinder
from pprint import pprint
tf.logging.set_verbosity(tf.logging.ERROR)
from random import choice
import gc
from bert import tokenization
from keras_xlnet.backend import keras
from keras_bert.layers import Extract
from keras_xlnet import Tokenizer, load_trained_model_from_checkpoint, ATTENTION_TYPE_BI

from google.colab import drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/Colab Notebooks/data")

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.14`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0



Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
np.random.seed(42)
tf.set_random_seed(42)

In [0]:
data_image = np.load("data_image_1.npy")
data_text = np.load("data_text_1.npy")
data_label = np.load("data_label_1.npy")
data_attribute = np.load('data_attribute_1.npy')
train_imagesX = np.load('train_imagesX_1.npy')
test_imagesX = np.load('test_imagesX_1.npy')

In [0]:
length = int(len(data_image) * 0.8)
train_images = data_image[:15360]
train_text = data_text[:15360]
train_attributes = data_attribute[:15360]
trainY = data_label[:15360]
test_images = data_image[15362:]
test_text = data_text[15362:]
test_attributes = data_attribute[15362:]
testY = data_label[15362:]

In [0]:
train_text = np.load('train_text.npy')
train_attribute = np.load('train_attribute.npy')
trainY = np.load('train_label.npy')
test_text = np.load('test_text.npy')
testY = np.load('test_label.npy')
test_attribute = np.load('test_attribute.npy')
visual_train = np.load('visual_features_train.npy')
visual_test = np.load('visual_features_test.npy')

train_text = train_text[:15344]
train_attributes = train_attribute[:15344]
trainY = trainY[:15344]
visual_train = visual_train[:15344]
test_text = test_text[:3824]
test_attributes = test_attribute[:3824]
testY = testY[:3824]
visual_test = visual_test[:3824]

In [0]:
EPOCH = 5
BATCH_SIZE = 16
SEQ_LEN = 196

In [8]:
checkpoint_path = 'chinese_xlnet_base_L-12_H-768_A-12'
in_attribute = keras.layers.Input(shape=(956,), name="attribute")
tokenizer = Tokenizer(os.path.join(checkpoint_path, 'spiece.model'))
xlnet = load_trained_model_from_checkpoint(
    config_path=os.path.join(checkpoint_path, 'xlnet_config.json'),
    checkpoint_path=os.path.join(checkpoint_path, 'xlnet_model.ckpt'),
    batch_size=BATCH_SIZE,
    memory_len=0,
    target_len=SEQ_LEN,
    in_train_phase=False,
    attention_type=ATTENTION_TYPE_BI,
)

text_output = xlnet.output
extract = Extract(index=-1, name='Extract')(text_output)
text_repr = keras.layers.Dense(128, activation='relu')(extract)


in_image = keras.layers.Input(shape=(1152,))
flat = keras.layers.Dense(768, activation='relu')(in_image)

visual_repr = keras.layers.Dense(128,activation='relu')(flat)

attribute_fc1 = keras.layers.Dense(512, activation='relu')
attribute_out_1 = attribute_fc1(in_attribute)
attribute_fc2 = keras.layers.Dense(512, activation='relu')(attribute_out_1)
att_repr = keras.layers.Dense(32,activation='relu')(attribute_fc2)

#classifier
combine_repr = keras.layers.concatenate([text_repr, visual_repr, att_repr])
com_drop=keras.layers.Dropout(0.3)(combine_repr)

com_drop = keras.layers.Dense(35, activation='relu')(com_drop)
com_drop = keras.layers.Dropout(0.3)(com_drop)

# extract = Extract(index=-1, name='Extract')(last)

# dense = keras.layers.Dense(units=768, name='Dense')(extract)

# norm = keras.layers.BatchNormalization(name='Normal')(dense)

output = keras.layers.Dense(1, activation='sigmoid')(com_drop)

model = keras.models.Model(inputs=[xlnet.input[0], xlnet.input[1], xlnet.input[2], in_image, in_attribute], outputs=output)

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 196)          0                                            
__________________________________________________________________________________________________
Embed-Token (EmbeddingRet)      [(None, 196, 768), ( 24576000    Input-Token[0][0]                
__________________________________________________________________________________________________
Masking (CreateMask)            (None, 196)          0           Input-Token[0][0]                
__________________________________________________________________________________________________
Embed-Token-Masked (RestoreMask (None, 196, 768)     0           Embed-Token[0][0]                
                                                                 Masking[0][0]              

In [0]:
def generate_sequence(text, images, attributes, Y):
    tokens, classes = [], []
    
    for i in text:
        encoded = tokenizer.encode(i)[:SEQ_LEN - 1]
        encoded = [tokenizer.SYM_PAD] * (SEQ_LEN - 1 - len(encoded)) + encoded + [tokenizer.SYM_CLS]
        tokens.append(encoded)

    tokens, classes = np.array(tokens), Y
    segments = np.zeros_like(tokens)
    segments[:, -1] = 1
    lengths = np.zeros_like(tokens[:, :1])

    return (np.array(tokens), np.array(segments), np.array(lengths), np.array(images), np.array(attributes), np.array(classes))


train_tokens, train_segments, train_lengths, train_images, train_attribute, trainy = generate_sequence(train_text, visual_train, train_attributes, trainY)

test_tokens, test_segments, test_lengths, test_images, test_attribute, testy = generate_sequence(test_text, visual_test, test_attributes, testY)

In [9]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00005),
    loss='binary_crossentropy',
    metrics=['accuracy'],
)

### 定义callback函数，只保留val_sparse_categorical_accuracy 得分最高的模型

from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("MMFFND-XLNet-Finetune.h5", monitor='accuracy', verbose=1, save_best_only=True,
                            mode='max')

model.fit(
    x=[train_tokens, train_segments, train_lengths, train_images, train_attribute], 
    y=trainy,
    batch_size=BATCH_SIZE,
    epochs=5,
    verbose=1,
    shuffle=False,
    validation_split=0.3,
    callbacks=[checkpoint]
)

Train on 10740 samples, validate on 4604 samples
Epoch 1/5
10740/10740 [==============================] - 409s 38ms/step - loss: 0.3152 - accuracy: 0.8719 - val_loss: 0.1492 - val_accuracy: 0.9500

Epoch 00001: accuracy improved from -inf to 0.87188, saving model to MMFFND-XLNet-Finetune.h5
Epoch 2/5
10740/10740 [==============================] - 391s 36ms/step - loss: 0.0988 - accuracy: 0.9654 - val_loss: 0.1574 - val_accuracy: 0.9672

Epoch 00002: accuracy improved from 0.87188 to 0.96536, saving model to MMFFND-XLNet-Finetune.h5
Epoch 3/5
10740/10740 [==============================] - 391s 36ms/step - loss: 0.0564 - accuracy: 0.9847 - val_loss: 0.1724 - val_accuracy: 0.9687

Epoch 00003: accuracy improved from 0.96536 to 0.98473, saving model to MMFFND-XLNet-Finetune.h5
Epoch 4/5
10740/10740 [==============================] - 391s 36ms/step - loss: 0.0223 - accuracy: 0.9939 - val_loss: 0.1345 - val_accuracy: 0.9728

Epoch 00004: accuracy improved from 0.98473 to 0.99395, saving mode

In [0]:
test_predict = model.predict([test_tokens, test_segments, test_lengths, test_images, test_attribute],batch_size=16)
test_predict = [1 if i>=0.5 else 0 for i in test_predict]

In [0]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
print(f'Accuracy == {accuracy_score(testY,test_predict)}')
print(f'F1 == {f1_score(testY,test_predict,average=None)}')
print(f'Precision == {precision_score(testY,test_predict,average=None)}')
print(f'Recall == {recall_score(testY,test_predict,average=None)}')

Accuracy == 0.9674479166666666
F1 == [0.96807152 0.96679947]
Precision == [0.94987469 0.98644986]
Recall == [0.98697917 0.94791667]


In [12]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
print(f'Accuracy == {accuracy_score(testY,test_predict)}')
print(f'F1 == {f1_score(testY,test_predict,average=None)}')
print(f'Precision == {precision_score(testY,test_predict,average=None)}')
print(f'Recall == {recall_score(testY,test_predict,average=None)}')

Accuracy == 0.9756799163179917
F1 == [0.97473513 0.97655659]
Precision == [0.96815974 0.98274987]
Recall == [0.98140044 0.97044088]


In [0]:
fault = []
for i in range(len(test_predict)):
    if test_predict[i] != testY[i]:
        fault.append(i)

In [0]:
len(fault)

105

In [0]:
for i in fault:
    print(test_text[i] + "  predict: " + str(test_predict[i]) + ", truth: " + str(int(testY[i])) + "\n")

这是我老家的房子，被泥石流攻击了一间，家具，被褥等都被埋了，房子已经成了危房，无法居住。现在整个村处于失联状态，有邻居走山路出来带出来的图片。所幸父亲平安。地处河北井陉县南障城镇大王帮村。希望政府给予帮扶！@河北电视台今日资讯@河北电视台@河北青年报@民生关注@石家庄的事儿  predict: 1, truth: 0

N多次余震，虽说我们昌吉无震感，但还是被吓到怀疑人生，天佑新疆四川震完新疆震，真的有些怕了四川地震后就有预感，半夜会不会地震，结果早上被晃醒，晃了应该得有一分钟，整个新疆都有震感，所以，天灾再次降临新疆，保佑新疆的所有人都平安无事  predict: 1, truth: 0

开点小工厂的真不容易，抗风险能力差，融资成本高，同是做实业的，只能跟做房地产的借高利贷……末了被人脱下裤子侮辱了，搭上儿子……媒体还要反转，你是非法老赖……浮世绘编年史  predict: 1, truth: 0

我太惨了，自从来了乡下，天天被蚊子咬被苍蝇追，吃不好还拉肚子，还要破财，最烦的是事情还没解决又不能走  predict: 1, truth: 0

在这个世界上别太依赖任何人，因为当你在黑暗中挣扎的时候，连你的影子也会离开你。————南京利刃私家侦探（私人侦探利刃）广州深圳珠海汕头佛山韶关河源惠州汕尾东莞江门阳江湛江茂名肇庆清远潮州云浮中山揭阳梅州太原大同阳泉长治晋城朔州晋中运城忻州临汾吕梁  predict: 1, truth: 0

【男子和女友ML时JJ自燃烧焦了】2人均是河南商丘当地一所学校的学生，女孩今年18。女孩透露爱爱前男友喜欢喝白酒，这次也是，正要到高潮的时候她突然觉得下体越来越热，而男友面部表情复杂说不出话，才发现有点不对。医生证实，男子JJ基本报废，女孩下体已被灼伤，目前起火原因不明。From陕西新闻网  predict: 0, truth: 1

分享图片草  predict: 0, truth: 1

方舟子是个什么玩意儿？  predict: 1, truth: 0

7月16日早上七点半左右在雁塔区西安外事学院东赢园雅筑西南坐出租车到陕西省人民医院急救中心门前下车，由于本人粗心大意将自己的紫色钱包落在出租车上，钱包内没多少钱，主要有自己好多证件，证件姓名晁娜，证件尾号7624望那位好心人看见能归还。谢谢@漂西安@漂西安@西安直播@西安城事

In [0]:
model.save("/content/gdrive/My Drive/Colab Notebooks/data/MMFFND-XLNet.h5")

In [0]:
bert = np.load("/content/gdrive/My Drive/Colab Notebooks/data/bert_predict.npy")

In [0]:
bert

array([[1.0000000e+00],
       [3.2156706e-05],
       [1.0000000e+00],
       ...,
       [7.3015690e-06],
       [5.3158402e-04],
       [1.0000000e+00]], dtype=float32)

In [0]:
xlnet = np.array(test_predict)

In [0]:
out = (bert[1:] + xlnet) / 2
out = [1 if i>=0.5 else 0 for i in out]

In [0]:
print(f'Accuracy == {accuracy_score(testY,out)}')
print(f'F1 == {f1_score(testY,out,average=None)}')
print(f'Precision == {precision_score(testY,out,average=None)}')
print(f'Recall == {recall_score(testY,out,average=None)}')

NameError: ignored